# Generate report page

DATA examples

In [ ]:
def make_mosaic(imgs, nrows, ncols, border=1):
    """Make a nice mosaic.
    
    Given a set of images with all the same shape, makes a
    mosaic with nrows and ncols;
    intended for use with activation layer
    """
    # set up data
    nimgs = imgs.shape[0]
    imshape = imgs.shape[1:]

    mosaic = np.ma.masked_all((nrows * imshape[0] + (nrows - 1) * border,
                           ncols * imshape[1] + (ncols - 1) * border),
                           dtype=np.float32)

    paddedh = imshape[0] + border
    paddedw = imshape[1] + border
    for i in xrange(nimgs):
        row = int(np.floor(i / ncols))
        col = i % ncols

        mosaic[row * paddedh:row * paddedh + imshape[0],
               col * paddedw:col * paddedw + imshape[1]] = imgs[i]
    return mosaic

CNN architecture vizualization

In [ ]:
# I did this in latex...

Loss/Accuracy plot

In [ ]:
#MINE
def loss_acc_plot(loss, val_loss, acc, val_acc, epochs):
# ------------------------------------------------------------------------------
# Funciton plots a combined loss and accuracy plot for training and validation set
# ------------------------------------------------------------------------------

    figsize=(6,4)
    fig, axis1 = plt.subplots(figsize=figsize)
    plot1_lacc = axis1.plot(epochs, acc, 'navy', label='accuracy')
    plot1_val_lacc = axis1.plot(epochs, val_acc, 'deepskyblue', label="validation accuracy")

    plot1_loss = axis1.plot(epochs, loss, 'red', label='loss')
    plot1_val_loss = axis1.plot(epochs, val_loss, 'lightsalmon', label="validation loss")


    plots = plot1_loss + plot1_val_loss
    labs = [l.get_label() for l in plots]
    axis1.set_xlabel('Epoch')
    axis1.set_ylabel('Loss/Accuracy')
    plt.tight_layout()
    axis1.legend(loc='center right')
    return

Confusion Matrix plot with original and normalized values

In [ ]:
#MINE
def CM_ROC(x_test,y_test):
# ------------------------------------------------------------------------------
# Evaluate classificaiton
# Outputs the confusion matrix and ROC curve vith its AUC 
# ------------------------------------------------------------------------------
    # predict
    prob = loaded_model.predict(x_test)
    pred =  (prob > 0.5).astype('int32') 

    # measure confusion
    labels=[0, 1]
    cm = metrics.confusion_matrix(y_test, pred, labels=labels)
    cm = cm.astype('float')
    cm_norm = cm / cm.sum(axis=1)[:, np.newaxis]
    print("cm", cm)
    print("cm_norm", cm_norm)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, prob, pos_label=1)
    auc = metrics.roc_auc_score(y_test, prob)
    print("AUC:", auc)

    #plotting
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Confusion matrix (Pristine Images)',y=1.08)
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    fmt = '.2f'
    thresh = cm_norm.max() / 2.
    for i in range(cm_norm.shape[0]):
        for j in range(cm_norm.shape[1]):
            ax.text(j, i, format(cm_norm[i, j], fmt),
            ha="center", va="center",
            color="white" if cm_norm[i, j] < thresh else "black")
    #pl.savefig('FINAL_SB00_Conf_Matrix_TEST_68_4_9.pdf')
    plt.show()
    
    #ROC
    figsize=(5,5)
    fig, axis1 = plt.subplots(figsize=figsize)
    x_onetoone = y_onetoone = [0, 1]

    plt.plot(fpr, tpr, 'r-')
    plt.plot(x_onetoone, y_onetoone, 'k--',  label="1-1")
    plt.legend(loc=0)
    plt.title("Receiver Operator Characteristic (ROC)")
    plt.xlabel("False Positive (1 - Specificity)")
    plt.ylabel("True Positive (Selectivity)")
    plt.tight_layout()
    #pl.savefig('FINAL_SB00_ROC_TEST_bestweights_68_4_9.pdf')
    return prob, pred, auc

In [ ]:
def plot_confusion_matrix(dir_analysis_figure, cm, label_quadrant=True, labels=[0,1]):
    """Plot the confusion matrix."""
    # check shape
    assert cm.shape == (2,2), "Not the right shape for input cm"

    sns.set_style("white")
    sns.set_style("ticks")
    n_colors=300
    # cmap = sns.cubehelix_palette(n_colors, start=2.8, rot=.1, as_cmap=True) #, start=1, rot=1.2, as_cmap=True)
    cmap = sns.color_palette("coolwarm", n_colors=n_colors)#, color_codes=True)
    cmap = ListedColormap(cmap)

    # plot
    plt.figure()
    plt.imshow(np.transpose(cm[::-1,::-1]), 
               interpolation='nearest', cmap=cmap, extent=[1.5,-0.5,-0.5,1.5])
    plt.title("Confusion Matrix")
    plt.xlabel('Predicted', fontsize=20)
    plt.ylabel('True', fontsize=20)
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=20) 
    plt.tight_layout()

    if label_quadrant:
        tp_x, tp_y = 1, 1
        tn_x, tn_y = 0, 0
        fn_x, fn_y = 0, 1
        fp_x, fp_y = 1, 0

        verticalalignment = 'center'
        myformat = '{:04.3f}'
        tp_str = "TP:\n" + myformat.format(cm[tp_y,tp_x])
        tn_str = "TN:\n" + myformat.format(cm[tn_y,tn_x])
        fp_str = "FP:\n" + myformat.format(cm[fp_y,fp_x])
        fn_str = "FN:\n" + myformat.format(cm[fn_y,fn_x])
        plt.text(tn_x, tn_y, tn_str,
                 fontsize=25, color='white', fontweight='bold',
                 verticalalignment=verticalalignment, 
                 horizontalalignment='center')
        plt.text(tp_x, tp_y, tp_str,
                 fontsize=25, color='white', fontweight='bold',
                 verticalalignment=verticalalignment,
                 horizontalalignment='center')
        plt.text(fp_x, fp_y, fp_str,
                 fontsize=25, color='white', fontweight='bold',
                 verticalalignment=verticalalignment,
                 horizontalalignment='center')
        plt.text(fn_x, fn_y, fn_str,
                 fontsize=25, color='white', fontweight='bold',
                 verticalalignment=verticalalignment,
                 horizontalalignment='center')

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        top='off',
        labelsize=20)         # ticks along the top edge are off
    plt.tick_params(
        axis='y',          
        which='both',      
        labelsize=20)      

    label_dict = {0: "Non", 1: "Lens"}
    labels_lab = [ label_dict[labels[0]], label_dict[labels[1]] ]  
    plt.xticks(labels, labels_lab)
    plt.yticks(labels, labels_lab)

    make_dir(dir_analysis_figure)
    file_save = dir_analysis_figure + "confusion.png"
    plt.savefig(file_save)

    return file_save

True vs prediction plot for all test examples

In [ ]:
def tru_pred (y_train, y_pred_train, y_valid, y_pred_valid, y_test, y_pred):
    fig, axis1 = plt.subplots(figsize=(8,8))
    plt.scatter(y_test, y_pred, label='comparison')
    plt.scatter(y_train, y_pred_train, s=3, label='train set')
    plt.scatter(y_valid, y_pred_valid, s=5, label='valid set')
    plt.plot([0,1], [0,1], 'k--', label="1-1")
    #plt.title("Receiver Operator Characteristic (ROC)")
    plt.xlabel("Truth")
    plt.ylabel("Prediction")
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig('regression_ytest'+cnn_label)
    return

ROC curve with AUC

In [ ]:
def plot_roc(dir_analysis_figure, fpr, tpr):
    """Plot the ROC."""
    # plot

    sns.set_style("white")
    sns.set_style("ticks")

    fig, axis1 = plt.subplots(figsize=(10,8))
    sns.set_palette("coolwarm", n_colors=6)
    sns.set_color_codes()
    lw = 3
    legend_label_size = "xx-large"

    plt.plot(fpr, tpr, 'r-', lw=lw, label="ROC curve")
    plt.plot([0, 1], [0, 1], 'k--', lw=lw, label="1-1")
    plt.xlabel("False positive", fontsize=25)
    plt.ylabel("True positive", fontsize=25)

    axis1.grid(False)
    axis1.tick_params(labeltop='off', labelright='off')
    axis1.xaxis.set_ticks_position('bottom')
    axis1.yaxis.set_ticks_position('left')

    legend1 = plt.legend(loc='lower right', shadow=True, prop={'size':25})
    for label in legend1.get_texts():
            label.set_fontsize(legend_label_size)
    for label in legend1.get_lines():
        label.set_linewidth(lw)  # the legend line width

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        top='off',
        labelsize=20)         # ticks along the top edge are off
    plt.tick_params(
        axis='y',          
        which='both',      
        labelsize=20)      

    # save to file
    # make output directory if necessary
    make_dir(dir_analysis_figure)
    file_save = dir_analysis_figure + "roc.png"
    plt.savefig(file_save)

    # auc = metrics.roc_auc_score(ytrue, yscore)
    # print "auc:", auc

    return file_save

Histogram of all output predictions

In [ ]:
def plot_probability_distribution(dir_analysis_figure, y_prob_set):
    """Plot history as a function of epoch."""
    sns.set_style("white")
    sns.set_style("ticks")

    fig, axis1 = plt.subplots(figsize=(10,8))

    n_colors = 4
    sns.cubehelix_palette(n_colors, start=2.8, rot=.1) #, start=1, rot=1.2, as_cmap=True)
    sns.set_palette("coolwarm", n_colors=n_colors )
    sns.set_color_codes()
    lw = 5
    legend_label_size = "xx-large"
    nbins = 7


    plt.title("Probability Distributions")
    plt.xlabel('probability', fontsize=20)
        

    axis1.grid(False)
    axis1.tick_params(labeltop='off', labelright='off')
    axis1.xaxis.set_ticks_position('bottom')
    axis1.yaxis.set_ticks_position('left')


    y_prob_all = y_prob_set['all']
    y_prob_tp = y_prob_set['tp']
    y_prob_fp = y_prob_set['fp']
    y_prob_tn = y_prob_set['tn']
    y_prob_fn = y_prob_set['fn']

    hist_all, bins_all = np.histogram(y_prob_all, bins=nbins)
    center_all = (bins_all[:-1] + bins_all[1:]) / 2

    hist_tp, bins_tp = np.histogram(y_prob_tp, bins=nbins)
    center_tp = (bins_tp[:-1] + bins_tp[1:]) / 2

    hist_fp, bins_fp = np.histogram(y_prob_fp, bins=nbins)
    center_fp = (bins_fp[:-1] + bins_fp[1:]) / 2

    hist_tn, bins_tn = np.histogram(y_prob_tn, bins=nbins)
    center_tn = (bins_tn[:-1] + bins_tn[1:]) / 2

    hist_fn, bins_fn = np.histogram(y_prob_fn, bins=nbins)
    center_fn = (bins_fn[:-1] + bins_fn[1:]) / 2

    plt.yscale('log')
    #plt.xscale('log')
    axis1.plot(center_all, hist_all,  lw=lw, color='k', ls=":", label="All")
    axis1.plot(center_tp, hist_tp,  lw=lw, ls='-', label="TP")
    axis1.plot(center_fp, hist_fp,  lw=lw, ls='-', label="FP")
    axis1.plot(center_tn, hist_tn, lw=lw, ls='--', label="TN")
    axis1.plot(center_fn, hist_fn,  lw=lw, ls='--', label="FN")

    legend1 = plt.legend(loc='upper left', shadow=True, prop={'size':25})
    for label in legend1.get_texts():
        label.set_fontsize(legend_label_size)
    for label in legend1.get_lines():
        label.set_linewidth(lw)  # the legend line width

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        top='off',
        labelsize=20)         # ticks along the top edge are off
    plt.tick_params(
        axis='y',          
        which='both',      
        labelsize=20)       

    make_dir(dir_analysis_figure)
    file_save = dir_analysis_figure + "probability_distribution.png"
    plt.savefig(file_save)

    return file_save

# Extra Plots

precision / recall / f1 / brier

In [ ]:
#not a plot but we need to print these out
def scores(y_test, probability, prediction):
# ------------------------------------------------------------------------------
# Evaluate classificaiton
# Outputs the accuracy, precision, recall, f1 score, brier score of classificaiton
# ------------------------------------------------------------------------------
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, prediction)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, prediction)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, prediction)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, prediction)
    print('F1 score: %f' % f1)
    #brier score
    br = brier_score_loss(y_test, probability)
    print('Brier score is: %f' % br)
    return accuracy, precision, recall, f1, br

How these change if the probability treshold fro class changes

In [ ]:
def get_F1(df, mass_thresh='2e14', xlim=None, method='cnn'):
    if xlim is None:
    xlim = (0.8, 0.999) if method == 'cnn' else (3., 30.)
    col = f'score_wdust (trained>{mass_thresh})' if method == 'cnn' else 'mf_peaksig'
    Fscore = lambda x: _get_Fbeta(df[f'Truth(>{mass_thresh})'], (df[col]>x).astype(int))
    f = plt.figure(figsize=(12, 4))
    x = np.linspace(xlim[0], xlim[1])
    y = [Fscore(xx) for xx in x]
    plt.scatter(x, y)
    plt.xlim(xlim)
    plt.xlabel('%s thres'%("CNN prob" if method == 'cnn' else "MF S/N"), fontsize='xx-large')
    plt.ylabel('F1_score', fontsize='xx-large')
    return

2D histogram

In [ ]:
def twoD_histogram (prob_all, prob_TP_TN, variable_all, variable_TP_TN):
    # Plot 2D histogram of the distribution of all future mergers vs TP future mergers 
    #(stellar mass and the output probability)
    sns.set_style("white")
    plt.ylabel('CNN Output')
    plt.xlabel('Stellar Mass')
    plt.xlim(9.4, 11.8)
    plt.xticks([9.5, 10, 10.5, 11, 11.5])
    sns.kdeplot(variable_TP_TN, prob_TP_TN, cmap="RdGy",  n_levels=10)
    sns.kdeplot(variable_all_P_N, prob_all_P_N, cmap="coolwarm", n_levels=10)

    r = sns.color_palette("RdGy")[0]
    b = sns.color_palette("coolwarm")[0]

    red_patch = mpatches.Patch(color=r, label='TP/TN')
    blue_patch = mpatches.Patch(color=b, label='all positives/negatives')
    plt.legend(handles=[red_patch,blue_patch],loc='lower right')
    plt.show()
    return

# Bootstraping and errors

bootstraped errors

In [ ]:
def bootstraping_scores (y_test, probability, prediction, boot_num, rand_seed):
    y_pred = probability
    y_true = y_test
    pred = prediction

    #print("Original ROC area: {:0.3f}".format(roc_auc_score(y_true, y_pred)))

    n_bootstraps = boot_num#1000
    rng_seed = rand_seed#3  # control reproducibility
    bootstrapped_roc_auc = []
    bootstrapped_accuracy = []
    bootstrapped_precision = []
    bootstrapped_recall = []
    bootstrapped_f1 = []
    bootstrapped_brier = []

    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(y_pred) - 1, len(y_pred))
        if len(np.unique(y_true[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        roc_auc = roc_auc_score(y_true[indices], y_pred[indices])
        bootstrapped_roc_auc.append(roc_auc)
        #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))
   
        score_acc = accuracy_score(y_true[indices], pred[indices])
        bootstrapped_accuracy.append(score_acc)
    
        score_precision = precision_score(y_true[indices], pred[indices])
        bootstrapped_precision.append(score_precision)
    
        score_recall = recall_score(y_true[indices], pred[indices])
        bootstrapped_recall.append(score_recall)
    
        score_f1 = f1_score(y_true[indices], pred[indices])
        bootstrapped_f1.append(score_f1)
    
        score_brier = brier_score_loss(y_true[indices], y_pred[indices])
        bootstrapped_brier.append(score_brier)
        
        
    return bootstrapped_roc_auc,bootstrapped_accuracy, bootstrapped_precision,bootstrapped_recall,bootstrapped_f1,bootstrapped_brier

In [ ]:
def confidence_interval (bootstrapped_score,score):
    sorted_scores = np.array(bootstrapped_score)
    sorted_scores.sort()

    # Computing the lower and upper bound of the 90% confidence interval
    # You can change the bounds percentiles to 0.025 and 0.975 to get
    # a 95% confidence interval instead.
    confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]

    confidence_lower1 = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper1 = sorted_scores[int(0.975 * len(sorted_scores))]

    print("Original ROC area: {:0.3f}".format(score))

    print("90% Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
        confidence_lower, confidence_upper))
    print("95% Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
        confidence_lower1, confidence_upper1))
    print("95% Errors are: [{:0.3f} , {:0.3}]".format(
        confidence_lower1-score, confidence_upper1-score),"\n")
    return

bootstraped histograms

In [ ]:
def bootstraped_hist (bootstrapped_roc_auc, bootstrapped_accuracy, bootstrapped_precision, bootstrapped_recall, bootstrapped_f1, bootstrapped_brier):
    plt.hist(bootstrapped_roc_auc, bins=50)
    plt.title('Histogram of the bootstrapped ROC AUC scores')
    plt.show()
    
    plt.hist(bootstrapped_accuracy, bins=50)
    plt.title('Histogram of the bootstrapped accuraciy scores')
    plt.show()

    plt.hist(bootstrapped_precision, bins=50)
    plt.title('Histogram of the bootstrapped precision scores')
    plt.show()

    plt.hist(bootstrapped_recall, bins=50)
    plt.title('Histogram of the bootstrapped recall scores')
    plt.show()

    plt.hist(bootstrapped_f1, bins=50)
    plt.title('Histogram of the bootstrapped F1 scores')
    plt.show()
    return

bootstraped ROC

In [ ]:
def roc_with_CI (y_test, prob, num_boot, rand_seed):

    y_pred = prob
    y_true = y_test

    fpr, tpr, thresholds = metrics.roc_curve(y_test, prob, pos_label=1)
    
    n_bootstraps = num_boot # 1000
    rng_seed = rand_seed #3  # control reproducibility
    bootstrapped_fpr = []
    bootstrapped_tpr = []


    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(y_pred) - 1, len(y_pred))
        if len(np.unique(y_true[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue
        fpr_score, tpr_score, thresholds_score = metrics.roc_curve(y_true[indices], y_pred[indices], pos_label=1)

        bootstrapped_fpr.append(fpr_score)
        bootstrapped_tpr.append(tpr_score)

        
    tprs = []
    base_fpr = np.linspace(0, 1, 1001)   
    for i in range(len(bootstrapped_fpr)):    
        tpr1 = interp(base_fpr, bootstrapped_fpr[i], bootstrapped_tpr[i])
        tpr1[0] = 0.0
        tprs.append(tpr1)

    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std

    #https://dfrieds.com/math/confidence-intervals   for 95%CI
    tprs_upper_95 = mean_tprs - 1.96*std 
    tprs_lower_95 = mean_tprs + 1.96*std
    
        
    #plot
    plt.figure(figsize=(6, 6))
    x_onetoone = y_onetoone = [0, 1]
    prist = plt.plot(fpr, tpr, 'navy', linewidth=2, label='pristine images')
    prist1 = plt.fill_between(base_fpr, tprs_lower_95, tprs_upper_95, color='deepskyblue', alpha=0.9, label='95%CI pristine images')
    line11 = plt.plot(x_onetoone, y_onetoone, 'k--',linewidth=1, label="1-1")

    plt.legend(loc='lower right')

    plt.xlabel("False Positive (1 - Specificity)")
    plt.ylabel("True Positive (Selectivity)")
    plt.tight_layout()
    return

# Generate report

In [ ]:
def create_report(params, params_model, dirs, figure_dict, data_analysis):
    """Create report."""
    # report filename
    filename_report = dirs['analysis'] + "report_" + str(params["id_model"]).zfill(3) + "_" + str(params["id_analysis"]).zfill(3)
    

    # Create Document
    # geometry_options = {"tmargin": "-2.875in", "oddsidemargin": "-.875in", "evensidemargin":"-.875in", 
    #                    "textwidth":"0.75in", "textheight": "2.75in"}
    #geometry_options = {"tmargin": "-.875in", "lmargin": "1.5cm", "footskip":"30pt", "textheight":""}
    #geometry_options = {"margin": "-0.5in"}

    # create document
    doc = Document() #geometry_options=geometry_options)

    doc.preamble.append(NoEscape(r'\usepackage{float}'))
    #doc.preamble.append(NoEscape(r'\extrafloats{100}'))

    # Display basic analysis parameters
    with doc.create(Section('Analysis Parameters')):
        with doc.create(Tabular('c|c|c')) as table:
            table.add_row(("name", "id number", 'number of objects'))
            table.add_hline()
            table.add_hline()
            table.add_row(("id_model", params["id_model"], '-'))
            table.add_hline()
            table.add_row(("id_analysis", params["id_analysis"], '-'))
            table.add_hline()
            table.add_row(("id_data_train", params["id_data_train"], params_model["nb_train"]))
            table.add_hline()
            table.add_row(("id_data_valid", params["id_data_valid"], params_model["nb_valid"]))
            table.add_hline()
            table.add_row(("id_data_test", params["id_data_test"], params["nb_test"]))
            table.add_hline()


    with doc.create(Section('Confusion Diagnostics')):
        with doc.create(Tabular('c|c')) as table:
            table.add_hline()
            table.add_hline()
            table.add_row(('AUC', data_analysis['roc']['auc']))
            table.add_hline()
            table.add_hline()
            table.add_row(('Quality', data_analysis['quality']))
            table.add_hline()
            table.add_hline()
            table.add_row(('Confusion Elements', ''))
            table.add_hline()
            table.add_hline()
            for key, value in data_analysis['cm_elem'].items():
                table.add_row((key,value))
                table.add_hline()

    # model parameters
    with doc.create(Section('Training Parameters')):
        with doc.create(Tabular('c|c')) as table:
            table.add_hline()
            table.add_hline()
            metadata = dm.load_metadata_log(dirs['learnlist'], id_model=params["id_model"])
            #print 'metadata type', isinstance(metadata, dict)
            for key, val in metadata.items():
                table.add_row((key,val))
                table.add_hline()

    doc.append(NewPage()) 

    # model parameters
    with doc.create(Section('Model Summary')):
        file_model_layers = dirs["model"] + "model_summary.txt"
        f = open(file_model_layers, 'r')
        x = f.read()
        f.close()
        doc.append(x)

    doc.append(NewPage()) 
    # Diagnostics Figures
    with doc.create(Section('Diagnostic Figures')):
        doc.append(NewPage()) 
        ifig = 0
        print "figures"
        for key, val in figure_dict.items():
            try:
                print ifig, key, val
                with doc.create(Figure(position='')) as temp_object:
                    temp_object.add_image(figure_dict[key]['filename'], width=figure_dict[key]['width'])
                    temp_object.add_caption(key)
                ifig+=1
            except:
                pass

            if ifig % 10 == 0:
                 doc.append(NoEscape(r'\clearpage'))
            #    doc.append(NewPage())
   


    # generate pdf
    doc.generate_pdf(filename_report, clean_tex=False)
    return filename_report + ".pdf"